In [17]:
import requests
import pandas as pd
import numpy as np
import gmaps
from config import gkey
import json
from pprint import pprint

gmaps.configure(api_key = gkey)

In [18]:
path = "../Output_Data/Weather_data.csv"

weather_df = pd.DataFrame(pd.read_csv(path))

weather_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,East London,ZA,-33.02,27.91,68.00,94,42,2.24
1,1,Saint-Philippe,RE,-21.36,55.77,80.60,83,75,19.46
2,2,Ushuaia,AR,-54.80,-68.30,53.60,71,75,4.70
3,3,Hithadhoo,MV,-0.60,73.08,83.93,69,1,12.46
4,4,Henties Bay,NaN,-22.12,14.28,61.00,96,0,1.99
...,...,...,...,...,...,...,...,...,...
559,559,Banjarmasin,ID,-3.32,114.59,87.35,67,78,9.46
560,560,Kungurtug,RU,50.60,97.52,21.90,80,31,3.83
561,561,Jimma,ET,7.67,36.83,69.71,69,59,1.77
562,562,Vestmannaeyjar,IS,63.44,-20.27,35.87,84,100,8.75


In [19]:
lat_lng = []
humid = []

# A loop to add geo coordinates multiple times depending on humidity 
for index, row in weather_df.iterrows():
    
    if row["Humidity"] >= 90:
    
        lat_lng.append((row["Lat"], row["Lng"]))
        
    elif row["Humidity"] >= 75:
    
        lat_lng.append((row["Lat"], row["Lng"]))
        
    elif row["Humidity"] >= 60:
    
        lat_lng.append((row["Lat"], row["Lng"]))
        
    elif row["Humidity"] >= 50:
    
        lat_lng.append((row["Lat"], row["Lng"]))
    
    else:
        lat_lng.append((row["Lat"], row["Lng"]))


                                        
# Plot Heatmap
figure_layout = {
    'width': '1000px',
    'height': '750px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

fig.add_layer(gmaps.heatmap_layer(lat_lng))
fig

Figure(layout=FigureLayout(border='1px solid black', height='750px', margin='0 auto 0 auto', padding='1px', wi…

In [20]:
weather_df = weather_df.loc[weather_df['Max Temp'] < 80]
weather_df = weather_df.loc[weather_df['Max Temp'] > 70]
weather_df = weather_df.loc[weather_df['Wind Speed'] < 10]
weather_df = weather_df.loc[weather_df['Cloudiness'] == 0]

weather_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
88,88,Andradina,BR,-20.90,-51.38,72.00,84,0,5.17
104,104,Barra de Tecoanapa,MX,16.52,-98.75,72.55,90,0,2.10
206,206,Anta,IN,25.15,76.30,79.00,12,0,3.42
284,284,Bembèrèkè,BJ,10.23,2.66,77.67,50,0,4.25
292,292,San Luis,AR,-33.30,-66.34,77.50,51,0,8.55
302,302,Kintampo,GH,8.06,-1.73,78.76,78,0,3.29
387,387,Koutiala,ML,12.39,-5.46,71.89,11,0,8.39


In [45]:
hotels = []
city = []
country = []
geocords = []

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in weather_df.iterrows():
    
    geocords.append((row["Lat"],row["Lng"]))
    city.append(row["City"])
    country.append(row["Country"])

    # Parameters dictionary
    params = {
        "location": f'{row["Lat"]}, {row["Lng"]}',
        "keyword": "hotel",
        "radius": 5000,
        "type": "hotel",
        "key": gkey
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    
#     pprint(json.dumps(response, indent=4,sort_keys=True))

    try:
        hotels.append(response['results'][0]['name'])
        print(response['results'][0]['name'])
    except:
        1==1

Savana Park Hotel
Hotel Garapacho
Bharti Hotel
Hotel Bio Guera
Hotel Premium Tower Suites San Luis
Nobel Hotel
Hotel Flasso 1
[(-20.9, -51.38), (16.52, -98.75), (25.15, 76.3), (10.23, 2.66), (-33.3, -66.34), (8.06, -1.73), (12.39, -5.46)]


In [46]:
hotel_info = pd.DataFrame({"City":city,"Country":country,"Hotel":hotels,"Geo":geocords})
hotel_info

,City,Country,Hotel,Geo
0,Andradina,BR,Savana Park Hotel,"(-20.9, -51.38)"
1,Barra de Tecoanapa,MX,Hotel Garapacho,"(16.52, -98.75)"
2,Anta,IN,Bharti Hotel,"(25.15, 76.3)"
3,Bembèrèkè,BJ,Hotel Bio Guera,"(10.23, 2.66)"
4,San Luis,AR,Hotel Premium Tower Suites San Luis,"(-33.3, -66.34)"
5,Kintampo,GH,Nobel Hotel,"(8.06, -1.73)"
6,Koutiala,ML,Hotel Flasso 1,"(12.39, -5.46)"


In [50]:
# Plot Heatmap
figure_layout = {
    'width': '1000px',
    'height': '750px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

fig.add_layer(gmaps.heatmap_layer(lat_lng))


# Add hotel markers
marker_layer = gmaps.marker_layer(hotel_info["Geo"])
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='750px', margin='0 auto 0 auto', padding='1px', wi…